In [ ]:
import os
import datetime
import firebirdsql
import pandas as pd
from queue import Queue
import concurrent.futures
from openpyxl import Workbook
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

In [ ]:
# ===================== Funções de Conexão =====================
def get_firebird_connection():
    # Ajuste os parâmetros conforme sua configuração, inclusive charset
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

In [ ]:
conn = get_firebird_connection()
cursor = conn.cursor()

In [ ]:
# ---------------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------------
START_DATE = datetime.datetime(2025, 4, 15)   # 15/04/2025
END_DATE   = datetime.datetime(2025, 5, 17)
ONE_YEAR   = datetime.timedelta(days=365)

# Nome do ficheiro de saída
OUTPUT_FILE = f"./arquivos/relatorio_clientes_{END_DATE.strftime('%Y-%m-%d')}-nao_ef.xlsx"

In [ ]:
# ---------------------------------------------------------------
# 1) CAPTURA DE VENDAS / OS NO PERÍODO
# ---------------------------------------------------------------
import numpy as np


SQL_PEDIDOS = """
    SELECT pv.CDCLIENTE,
           c.NOME,
           'PEDIDOVENDA'        AS DOC_TYPE,
           pv.CDPEDIDOVENDA     AS DOC_NUM,
           pv.DATA              AS DATA_DOC,
           pv.VALORCDESC        AS VALOR
    FROM   PEDIDOVENDA pv
    JOIN   CLIENTE c ON c.CDCLIENTE = pv.CDCLIENTE
    WHERE  pv.EFETIVADO = 'N'
      AND (pv.DEVOLVIDO IS NULL OR pv.DEVOLVIDO <> 'S')
      AND  pv.CDCLIENTE <> 230
           AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
      AND  pv.DATA BETWEEN ? AND ?
"""

SQL_OS = """
    SELECT os.CDCLIENTE,
           c.NOME,
           'ORDEMSERVICO'       AS DOC_TYPE,
           os.CDORDEMSERVICO    AS DOC_NUM,
           os.DATA              AS DATA_DOC,
           os.VALORCDESC        AS VALOR
    FROM   ORDEMSERVICO os
           JOIN CLIENTE c ON c.CDCLIENTE = os.CDCLIENTE
    WHERE  os.CDCLIENTE <> 230
           AND UPPER(c.NOME) NOT LIKE '%COMAGRO%'
           AND os.DATA BETWEEN ? AND ?
            
"""

purchases_period = pd.concat([
    pd.read_sql(SQL_PEDIDOS, conn, params=(START_DATE, END_DATE)),
    pd.read_sql(SQL_OS,      conn, params=(START_DATE, END_DATE))
]).reset_index(drop=True)

# ---------------------------------------------------------------
# 2) ÚLTIMA COMPRA ANTES DE 15/04/2025 → detectar hiato > 1 ano
# ---------------------------------------------------------------
SQL_ULT_COMPRA = """
SELECT CDCLIENTE, MAX(DATA) AS ULT_COMPRA
FROM (
    SELECT CDCLIENTE, DATA FROM PEDIDOVENDA
     WHERE EFETIVADO='S' AND (DEVOLVIDO IS NULL OR DEVOLVIDO <> 'S')
    UNION ALL
    SELECT CDCLIENTE, DATA FROM ORDEMSERVICO
) t
WHERE DATA < ?
GROUP BY CDCLIENTE
"""

ult_compra_df = pd.read_sql(SQL_ULT_COMPRA, conn, params=(START_DATE, ), parse_dates=['ULT_COMPRA'])

# ---------------------------------------------------------------
# 3) METADADOS DOS CLIENTES (DATA DE CADASTRO)
# ---------------------------------------------------------------
clientes_df = pd.read_sql("SELECT CDCLIENTE, NOME, DTCADASTRO FROM CLIENTE", conn, parse_dates=['DTCADASTRO'])

# ---------------------------------------------------------------
# 4) PREPARAR BASE ÚNICA
# ---------------------------------------------------------------
# merge info cadastro / ult_compra → purchases_period
base = (purchases_period
        .merge(clientes_df, on=['CDCLIENTE', 'NOME'])
        .merge(ult_compra_df, on='CDCLIENTE', how='left'))

# garante que colunas de data estejam como datetime64[ns]
base[['DATA_DOC', 'DTCADASTRO', 'ULT_COMPRA']] = base[['DATA_DOC', 'DTCADASTRO', 'ULT_COMPRA']].apply(pd.to_datetime)

# Ordena inicialmente por nome + data (será reordenado mais adiante)
base = base.sort_values(['NOME', 'DATA_DOC']).reset_index(drop=True)

# ------------------------------------------------------------------
# 5) CLASSIFICAÇÃO DOS CLIENTES
#   • NOVO_CLIENTE  → nunca comprou antes de 15/04/2025
#   • REATIVADO>1Y → >1 ano sem comprar e voltou
#   • VELHO_ATIVO  → cadastro ≥1 ano e mantinha compras (<1 ano de hiato)
#   • RECENTE_ATIVO→ cadastro <1 ano antes de 15/04/2025 e já comprava
# ------------------------------------------------------------------

conds = [
    base['ULT_COMPRA'].isna(),                                                         # NOVO_CLIENTE
    base['ULT_COMPRA'].notna() & (base['ULT_COMPRA'] <= END_DATE - ONE_YEAR),          # REATIVADO>1Y
    base['DTCADASTRO'] <= (END_DATE - ONE_YEAR)                                        # VELHO_ATIVO
]
choices = ['NOVO_CLIENTE', 'REATIVADO>1Y', 'VELHO_ATIVO']
base['CATEGORY'] = np.select(conds, choices, default='RECENTE_ATIVO')


# ------------------------------------------------------------------
# 6) EXPORTAR PARA EXCEL
# ------------------------------------------------------------------
legend_df = pd.DataFrame({
    'CATEGORY': ['NOVO_CLIENTE', 'REATIVADO>1Y', 'VELHO_ATIVO', 'RECENTE_ATIVO'],
    'DESCRICAO': [
        'Primeira compra apenas a partir de 15/04/2025 (sem compras anteriores).',
        'Ficou >1 ano sem comprar e voltou a comprar.',
        'Cadastro ≥1 ano e continuou comprando (sem hiato >1 ano).',
        'Cadastro <1 ano antes de 15/04/2025 e já comprava antes.'
    ]
})

with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
    # Aba Dados (todos registros)
    base.to_excel(writer, sheet_name='Dados', index=False)

    # Legenda em K2:L5
    legend_df.to_excel(writer, sheet_name='Dados', index=False, startrow=1, startcol=10)  # col 10 -> K

    # Abas por categoria, sempre ordenadas
    for cat, df_cat in base.groupby('CATEGORY'):
        df_cat.sort_values(['NOME', 'DATA_DOC']).to_excel(writer, sheet_name=cat[:31], index=False)

    # Resumo geral
    resumo = (base.groupby('CATEGORY')
              .agg(Clientes_Unicos=('CDCLIENTE', pd.Series.nunique),
                   Qtde_Docs=('DOC_NUM', 'count'),
                   Valor_Total=('VALOR', 'sum'))
              .reset_index())
    resumo.to_excel(writer, sheet_name='Resumo_Geral', index=False)

    # -------------------------------------------------------------
    # FORMATAÇÃO (datas DD/MM/YY, moeda brasileira)
    wb = writer.book
    date_fmt = 'DD/MM/YY'
    currency_fmt = 'R$ #.##0,00'

    for ws in wb.worksheets:
        # Identifica cabeçalhos para aplicar formatação
        headers = {cell.value: cell.column for cell in ws[1]}
        max_row = ws.max_row

        for header, col_idx in headers.items():
            col_letter = ws.cell(row=1, column=col_idx).column_letter
            if header in ('DATA_DOC', 'DTCADASTRO', 'ULT_COMPRA'):
                for cell in ws[col_letter][1:max_row]:  # ignora cabeçalho
                    if isinstance(cell.value, (datetime.datetime, pd.Timestamp)):
                        cell.number_format = date_fmt
            if header == 'VALOR' or header == 'Valor_Total':
                for cell in ws[col_letter][1:max_row]:
                    if isinstance(cell.value, (int, float)):
                        cell.number_format = currency_fmt

        # Auto-ajuste de largura
        for col in ws.columns:
            max_len = max(len(str(cell.value)) if cell.value is not None else 0 for cell in col)
            ws.column_dimensions[col[0].column_letter].width = max_len + 2

print(f"Relatório gerado em: {OUTPUT_FILE}")

In [ ]:

query = """
            SELECT NOME, OBS
            FROM CLIENTE
            ORDER BY NOME
        """
        
cursor.execute(query)
rows = cursor.fetchall()

In [ ]:
import re

def sanitize_string(value):
    """
    Remove caracteres ilegais para XML (usado pelo OpenPyXL) de uma string.
    """
    # Essa expressão regular remove caracteres com códigos 0-8, 11, 12 e 14-31.
    illegal_xml_chars_re = re.compile(r'[\000-\010\013\014\016-\037]')
    return illegal_xml_chars_re.sub("", value)

In [ ]:
wb = Workbook()
ws = wb.active

for cliente in rows:
    processed_row = []
    for column in cliente:
        # Se for do tipo bytes, decodifica-o para string
        if isinstance(column, bytes):
            try:
                decoded = column.decode("iso-8859-1", errors="replace")
            except UnicodeDecodeError:
                decoded = column.decode("utf-8", errors="replace")
            processed_row.append(sanitize_string(decoded))
        else:
            processed_row.append(column)
    # Adiciona a linha completa ao worksheet
    ws.append(processed_row)

# Salva o arquivo Excel
output_filename = "clientes-obs.xlsx"
wb.save(output_filename)
print(f"Arquivo Excel salvo como '{output_filename}'")

In [ ]:
# ===================== Criação do Excel =====================
wb = Workbook()
ws = wb.active

# Definindo o header na ordem especificada
header = [
    "CDCLIENTE",
    "NOME",
    "DESCONTOHBL",
    "ENDERECO",
    "BAIRRO",
    "CIDADE",
    "ESTADO",
    "CEP",
    "VALORAFATURAR",
    "VALORFATURADO",
    "DTCADASTRO",
    "DTULTIMOMOV",
    "TEMENDENVIO",
    "TEMENDNF",
    "PRAZOPGTO",
    "CDTIPO",
    "CDDESCONTO",
    "TIPO",
    "CDLOCAL",
    "CDLOCALCOBRANCA",
    "PRAZOPEDIDO",
    "LIMITECREDITO",
    "SITUACAO",
    "CPF_CNPJ",
    "NOMEFANTASIA",
    "DTFUNDACAO",
    "EMAIL",
    "INSCRICAOMUNICIPAL",
    "RESPONSAVELPGTO",
    "FONERESPPGTO",
    "RESPONSAVELCOMPRA",
    "FONERESPCOMPRA",
    "NUMREGJUNTACOMERCIAL",
    "FILIACAPAI",
    "FILIACAOMAE",
    "DTNASCIMENTO",
    "IE",
    "UF_RG",
    "OBS",
    "INFESCOLHIDA",
    "CDATIVIDADECOMERCIAL",
    "CDAGENCIAPUBL",
    "PORCENTAGEM_AGENCIA",
    "TEMP_VALDUPLICATAS",
    "TEMP_VALDEBITO",
    "DIASPRAZOPAG",
    "CDTIPONFSAIDA",
    "CDFUNC",
    "CDMUNIC",
    "ENDER_NUMERO",
    "PONTORF",
    "APELIDO",
    "PAF_CODIGO_CLIENTE_COMPOSTO",
    "PAF_CODIGO_EMPRESA",
    "PAF_MODO_BD",
    "PAF_CPF",
    "PAF_SALDO_DISPONIVEL",
    "RELACIONAMENTOATIVO",
    "RELACIONAMENTOVALIDADE",
    "CDCLIENTEINDICACAO",
    "CADASTROCOMPLETO",
    "HABILITAFORMAPAG",
    "TEMPOATUACAORAMO",
    "CONSUMIDORFINAL",
    "DTULTICONTATO"
]

# Insere o header como primeira linha do Excel
ws.append(header)

# Insere as linhas de dados (cada linha já com 38 colunas)
# Para cada registro (cliente), monta uma linha processada
for cliente in rows:
    processed_row = []
    for column in cliente:
        # Se for do tipo bytes, decodifica-o para string
        if isinstance(column, bytes):
            try:
                decoded = column.decode("iso-8859-1", errors="replace")
            except UnicodeDecodeError:
                decoded = column.decode("utf-8", errors="replace")
            processed_row.append(sanitize_string(decoded))
        else:
            processed_row.append(column)
    # Adiciona a linha completa ao worksheet
    ws.append(processed_row)

# Salva o arquivo Excel
output_filename = "clientes.xlsx"
wb.save(output_filename)
print(f"Arquivo Excel salvo como '{output_filename}'")

In [ ]:
# Definindo o período: de 01/03/2024 a 28/02/2025
start_date = datetime.date(2024, 3, 1)
end_date = datetime.date(2025, 2, 28)

# Conectando e executando a query para extrair os dados da tabela APAGAR
conn = get_firebird_connection()
query = """
    SELECT DTVENCIMENTO, VALOR
    FROM APAGAR
    WHERE DTVENCIMENTO BETWEEN ? AND ?
"""
# Lê os dados da query para um DataFrame
df = pd.read_sql(query, conn, params=(start_date, end_date))
conn.close()

# Converte a coluna DTVENCIMENTO para datetime (assumindo que as datas estejam no formato dd/mm/yy)
df['DTVENCIMENTO'] = pd.to_datetime(df['DTVENCIMENTO'], dayfirst=True)

# Cria colunas auxiliares para o agrupamento:
# 'year_month' conterá o período do mês (ex.: 2024-03)
# 'day' conterá o dia do mês (1,2,...,31)
df['year_month'] = df['DTVENCIMENTO'].dt.to_period('M')
df['day'] = df['DTVENCIMENTO'].dt.day

# Agrupa por 'year_month' e 'day', calculando a média do VALOR para cada dia
grouped = df.groupby(['year_month', 'day'])['VALOR'].mean().reset_index()
grouped = grouped.rename(columns={'VALOR': 'avg_valor'})

# Para cada mês, seleciona o dia com a média de gasto mais baixa
result = grouped.loc[grouped.groupby('year_month')['avg_valor'].idxmin()].reset_index(drop=True)

# Exibe o resultado
print("Dia de menor gasto (média) por mês:")
print(result)